**Connect google drive**

In [1]:
# Check if the NVIDIA GPU is enabled or not.   
!nvidia-smi

Wed Mar 10 14:15:04 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    31W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [19]:
#mounting google drive, so as to use yolov3 folder that contains images for training in zip format.
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive

Mounted at /content/gdrive
ln: failed to create symbolic link '/mydrive/My Drive': File exists


**1) Clone the Darknet**



In [4]:
#As the darknet implementation is present on the github, so cloning darknet using '!git' command.
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 14730, done.
remote: Total 14730 (delta 0), reused 0 (delta 0), pack-reused 14730
Receiving objects: 100% (14730/14730), 13.27 MiB | 17.32 MiB/s, done.
Resolving deltas: 100% (10020/10020), done.


**2) Compile Darknet using Nvidia GPU**


In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
#Change makefile to have GPU and OPENCV enabled
#As we are using NVIDIA GPU, we have to make some changes in the darknet file.
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile 
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make #this will apply the changes in the entire darknet file.
  
#sed : stream editor, searching, find and replace, insertion or deletion, basically used to edit the content of the file.
#-i : used here to append the file.
#make :  command used to read all the editted lines from the 'Makefile', also the compilation of code after reading is done by this command.

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [7]:
#Making a copy of 'yolov3.cfg' file, named as 'yolov3_training.cfg' and making some changes in that file.(In cfg folder inside the whole darknet file)
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

#It is a configuration file for training which contains all the variable required to train the network like, batch_size, subdivision, max_batches
#activation functions, etc.

In [8]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

#4000 is the total batches
#if there are 3 classes, 2000 batches is sufficient for one class
#(means 2000 iterations for each class).    
#so, total batches in that case will be 6000.

- 'batch size = 1 to batch size = 64' in cfg/yolov3_training.cfg
- 'subdivisions = 1 to subdivisions = 16' in cfg/yolov3_training.cfg
- 'max_batches = 500200 to max_batches = 4000' in cfg/yolov3_training.cfg
- 'At line 610, classes = 80 to classes = 1' in cfg/yolov3_training.cfg
- Formula for no. of filters to be used:
- (no. of classes + 5)*3

In [9]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"
#DO NOT need to execute this as we have created the folder in gdrive containing images. 

mkdir: cannot create directory ‘/mydrive/yolov3’: Transport endpoint is not connected


In [10]:
!rm -rf data
#removing the folder named 'data', if it is already present inside the darknet folder.

In [11]:
!mkdir data
#creating a folder named data inside the darknet folder.

In [12]:
!echo "Number_Plates" > data/obj.names
#printing our class name i.e, 'Koala' into the file 'obj.names' in darknet/data/obj folder.

!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
#All these info will be written in the file 'obj.data' in darknet/data/obj folder.

!mkdir data/obj
#Creating folder obj in darknet/data folder.

- classes= 1
- train data from - data/train.txt
- validation data from - data/test.txt
- class names from - data/obj.names
- backup = /mydrive/yolov3

In [13]:
# Downloading weights of the layers of darknet model 53 from this link.
!wget https://pjreddie.com/media/files/darknet53.conv.74

#wget command used to get data from online source.

--2021-03-10 14:24:37--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  23.2MB/s    in 7.5s    

2021-03-10 14:24:45 (20.8 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [20]:
#Extracting images present in the images.zip archive, which is itself inside the 'yolov3' folder on google drive. And all the images will be saved into obj
#directory in data folder, that we just created.
!unzip /mydrive/yolov3/images.zip -d data/obj
#!unrar x /mydrive/yolov3/images.rar  data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/9.jpg          
  inflating: data/obj/9.txt          
  inflating: data/obj/10.jpg         
  inflating: data/obj/10.txt         
  inflating: data/obj/11.jpg         
  inflating: data/obj/11.txt         
  inflating: data/obj/12.jpg         
  inflating: data/obj/12.txt         
  inflating: data/obj/13.jpg         
  inflating: data/obj/13.txt         
  inflating: data/obj/14.jpg         
  inflating: data/obj/14.txt         
  inflating: data/obj/15.jpg         
  inflating: data/obj/15.txt         
  inflating: data/obj/16.jpg         
  inflating: data/obj/16.txt         
  inflating: data/obj/17.jpg         
  inflating: data/obj/17.txt         
  inflating: data/obj/18.jpg         
  inflating: data/obj/18.txt         
  inflating: data/obj/19.jpg         
  inflating: data/obj/19.txt         
  inflating: data/obj/20.jpg         
  inflating: data/obj/20.txt         
  inflating: data/obj/21.jpg         
  inflating: 

In [21]:
#displaying list of images.
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/112.jpg', 'data/obj/14.jpg', 'data/obj/116.jpg', 'data/obj/104.jpg', 'data/obj/46.jpg', 'data/obj/217.jpg', 'data/obj/170.jpg', 'data/obj/19.jpg', 'data/obj/198.jpg', 'data/obj/151.jpg', 'data/obj/113.jpg', 'data/obj/221.jpg', 'data/obj/167.jpg', 'data/obj/95.jpg', 'data/obj/11.jpg', 'data/obj/137.jpg', 'data/obj/196.jpg', 'data/obj/36.jpg', 'data/obj/100.jpg', 'data/obj/136.jpg', 'data/obj/144.jpg', 'data/obj/152.jpg', 'data/obj/148.jpg', 'data/obj/220.jpg', 'data/obj/134.jpg', 'data/obj/39.jpg', 'data/obj/63.jpg', 'data/obj/60.jpg', 'data/obj/184.jpg', 'data/obj/4.jpg', 'data/obj/139.jpg', 'data/obj/98.jpg', 'data/obj/234.jpg', 'data/obj/75.jpg', 'data/obj/115.jpg', 'data/obj/81.jpg', 'data/obj/90.jpg', 'data/obj/101.jpg', 'data/obj/183.jpg', 'data/obj/61.jpg', 'data/obj/146.jpg', 'data/obj/8.jpg', 'data/obj/161.jpg', 'data/obj/5.jpg', 'data/obj/43.jpg', 'data/obj/120.jpg', 'data/obj/155.jpg', 'data/obj/45.jpg', 'data/obj/21.jpg', 'data/obj/57.jpg', 'data/obj/179.jpg', 'da

In [22]:
#Create training.txt file
file = open("data/train.txt", "w")    #same file name which is mentioned in cell[8].
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:
#Training the network using, the updated training configuration file i.e, 'training.cfg' and darknet weights file downloaded through that link.
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
 total_bbox = 172231, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.913293), count: 4, class_loss = 0.000076, iou_loss = 0.061629, total_loss = 0.061705 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 172235, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.969661), count: 4, class_loss = 0.000005, iou_loss = 0.006183, total_loss = 0.006188 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (ms

This network creates a checkpoint after every 1000 epochs.
At last, 'last weights' file will be created which we have to use for making predictions.